In [2]:
import requests
import pandas as pd
import time
import numpy as np
import re
import sys
import random


In [3]:
jeju_food = pd.read_csv('../data/공공데이터/제주특별자치도음식점목록(통합).csv')

In [4]:
jeju_food.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42602 entries, 0 to 42601
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   사업장명     42602 non-null  object 
 1   업종구분대분류  42602 non-null  object 
 2   업종구분소분류  42599 non-null  object 
 3   인허가일자    42584 non-null  float64
 4   인허가취소일자  18 non-null     object 
 5   영업상태명    42602 non-null  object 
 6   상세영업상태명  42594 non-null  object 
 7   폐업일자     23435 non-null  float64
 8   휴업시작일자   0 non-null      float64
 9   휴업종료일자   0 non-null      float64
 10  재개업일자    18 non-null     float64
 11  소재지면적    41998 non-null  object 
 12  소재지전체주소  42420 non-null  object 
 13  도로명전체주소  27767 non-null  object 
 14  도로명우편번호  27433 non-null  object 
 15  데이터갱신일자  42584 non-null  object 
dtypes: float64(5), object(11)
memory usage: 5.2+ MB


In [5]:
jeju_food_1=jeju_food[jeju_food['영업상태명'] != '폐업']

In [6]:
jeju_food_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19157 entries, 0 to 41281
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   사업장명     19157 non-null  object 
 1   업종구분대분류  19157 non-null  object 
 2   업종구분소분류  19155 non-null  object 
 3   인허가일자    19149 non-null  float64
 4   인허가취소일자  8 non-null      object 
 5   영업상태명    19157 non-null  object 
 6   상세영업상태명  19149 non-null  object 
 7   폐업일자     0 non-null      float64
 8   휴업시작일자   0 non-null      float64
 9   휴업종료일자   0 non-null      float64
 10  재개업일자    8 non-null      float64
 11  소재지면적    19148 non-null  object 
 12  소재지전체주소  19062 non-null  object 
 13  도로명전체주소  18925 non-null  object 
 14  도로명우편번호  18818 non-null  object 
 15  데이터갱신일자  19149 non-null  object 
dtypes: float64(5), object(11)
memory usage: 2.5+ MB


In [7]:
jeju_food_1.reset_index(inplace=True)

In [8]:
jeju_food_1.drop('index',axis=1,inplace=True)

C:\Users\TJ\AppData\Local\Temp\ipykernel_7244\4260925088.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jeju_food_1.drop('index',axis=1,inplace=True)


In [9]:
# 1차 검색 제주 000 
url = 'https://map.naver.com/v5/api/search'
params = {
    'calller': 'pcweb',
    'query': '',
    'type': 'all',
    'searchCoord': '126.56379330000001;33.24652849999992',
    'page': 1,
    'displayCount': 20,
    'isPlaceRecommendationReplace': True,
    'lang': 'ko'
}
tmp_dict = dict()
headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'ko-KR,ko;q=0.8,en-US;q=0.6,en;q=0.4',
    'cache-control': 'no-cache',
    'content-type': 'application/json',
    'cookie': 'NNB=SFVP2LHVPD7WC; NDARK=N; _gcl_au=1.1.876992382.1652928740; _fbp=fb.1.1652928739593.996991756; _ga=GA1.2.1723077753.1655082737; MM_NEW=1; NFS=2; _tt_enable_cookie=1; _ttp=d30a8fad-c361-4bb2-a6d7-884e4f9ca1c6; NV_WETR_LOCATION_RGN_M="MDkxNDAxMDQ="; NV_WETR_LAST_ACCESS_RGN_M="MDkxNDAxMDQ="; nx_ssl=2; csrf_token=b22420dac145f9757a431121f5b4c25948818847853c526728a4d2219155c97e9b6490d088524f5e2d1f9fdd189f23d95bdd86f6e9ca8c8b07bf30c857e040c6; page_uid=d78aa3bb-613a-4eab-8035-b9c4712fa1de',
    'expires': 'Sat, 01 Jan 2000 00:00:00 GMT',
    'pragma': 'no-cache',
    'referer': 'https://map.naver.com/',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'
}
params2 ={
    'lang' :'ko'
}
#n차 검색 제주도 주소 검색 후 해당 위치의 가게 정보를 조회 
url3 = 'https://map.naver.com/v5/api/addresses/place/'
params3= {
    'address':'',
    'lang':'ko',
    'page':1
}
headers2 = {
    'authority': 'map.naver.com',
    'accept': 'application/json, text/plain, */*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'ko-KR,ko;q=0.8,en-US;q=0.6,en;q=0.4',
    'cache-control': 'no-cache',
    'content-type': 'application/json',
    'cookie': 'NNB=SFVP2LHVPD7WC; NDARK=N; _gcl_au=1.1.876992382.1652928740; _fbp=fb.1.1652928739593.996991756; _ga=GA1.2.1723077753.1655082737; MM_NEW=1; NFS=2; _tt_enable_cookie=1; _ttp=d30a8fad-c361-4bb2-a6d7-884e4f9ca1c6; NV_WETR_LOCATION_RGN_M="MDkxNDAxMDQ="; NV_WETR_LAST_ACCESS_RGN_M="MDkxNDAxMDQ="; nx_ssl=2; page_uid=hXuWqwp0YiRssBM4BUGssssssMo-163853; BMR=s=1658907970093&r=https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=heygun&logNo=221520454294&r2=https://www.google.com/; page_uid=3c7d105d-de2b-4c43-8a74-1c2abebf354d',
    'expires': 'Sat, 01 Jan 2000 00:00:00 GMT',
    'pragma ': 'no-cache',
    'referer': 'https://map.naver.com/',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'
}

In [10]:
title=jeju_food_1['사업장명']

In [11]:
title

0               카페송키
1        그림책방앤카페노란우산
2           에릭스에스프레소
3           일품순두부한림점
4           팔도실비집아라점
            ...     
19152            쏭카롱
19153      주식회사 애니멀고
19154         한옥마루카페
19155    쎄븐일레븐서귀포법환점
19156           신신카페
Name: 사업장명, Length: 19157, dtype: object

In [25]:
restaurant_result = []
none_lst = []
for num,v in enumerate(title):
    searchword = '제주 '+  v
    #정규표현식으로 ( ) 안 텍스트 전부 삭제
    rmve_bracket = r'\([^)]*\)'
    searchword = re.sub(rmve_bracket,'',searchword)
    params['query'] = searchword
    time.sleep(random.uniform(1.5,2.5))
    response1 = requests.get(url,params,headers=headers)
    if response1.status_code == 200:
        j_text = response1.json()
    else:
        print(f'{num}번째 실패')
        sys.exit('response1.status_code 조회해보기')
        
    #장소정보 저장    
    tmp_places = j_text['result']['place']    
    search_bool = False
    #검색결과 없을 때
    if tmp_places is None:
        none_lst.append(title[num])
        print(f'{num} 번째 {title[num]} 조회 결과 존재 X')
        continue
    #검색 결과 존재시    
    else:
        tmp_list=tmp_places['list']
        if tmp_list is None:
            none_lst.append(title[num])
            print(f'{num} 번째 {title[num]} 조회 결과 존재 X')
            continue
        else:
            
            #검색결과가 하나일 때 바로 조회
            if len(tmp_list) == 1:
                tmp_id = tmp_list[0]['id']
                url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
                time.sleep(random.uniform(1.5,2.5))
                response2 = requests.get(url2,params2,headers=headers2)
                if response2.status_code == 200:
                    tmp_dict = response2.json()
                    restaurant_result.append(tmp_dict)
                    search_bool = True
                    print(f'{num}번째 {searchword} 중 {tmp_id}의 1개 요약본 저장 완료')
                else:
                    print(f'{num}번째 {searchword}의 {tmp_id}로 1개 요약본 가져오기 실패')
                    sys.exit('response2.status_code 조회해보기')
                continue
            #검색결과 여러개일 때    
            else:
                for index, lst in enumerate(tmp_list):
                    tmp_categories =lst['category']
                    for category in tmp_categories:
                        if '식' in category:
                            tmp_id = lst['id']
                            #주위 가게로 가져온 id 값으로 요약본 출력
                            url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
                            time.sleep(random.uniform(1.5,2.5))
                            response2 = requests.get(url2,params2,headers=headers2)
                            if response2.status_code == 200:
                                tmp_dict = response2.json()
                                restaurant_result.append(tmp_dict)
                                search_bool = True
                                print(f'{num}번째 {searchword} 중 {tmp_id}의 식당 요약본 저장 완료')
                            else:
                                print(f'{num}번째 {searchword}의 {tmp_id}로 식당 요약본 가져오기 실패')
                                sys.exit('response2.status_code 조회해보기')
                            break
                        elif '술집' in category:
                            tmp_id = lst['id']
                            #주위 가게로 가져온 id 값으로 요약본 출력
                            url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
                            time.sleep(random.uniform(1.5,2.5))
                            response2 = requests.get(url2,params2,headers=headers2)
                            if response2.status_code == 200:
                                tmp_dict = response2.json()
                                restaurant_result.append(tmp_dict)
                                search_bool = True
                                print(f'{num}번째 {searchword} 중 {tmp_id}의  술집 요약본 저장 완료')
                            else:
                                print(f'{num}번째 {searchword}의 {tmp_id}로 술집 요약본 가져오기 실패')
                                sys.exit('response2.status_code 조회해보기')
                            break
                        elif '밥' in category:
                            tmp_id = lst['id']
                            #주위 가게로 가져온 id 값으로 요약본 출력
                            url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
                            time.sleep(random.uniform(1.5,2.5))
                            response2 = requests.get(url2,params2,headers=headers2)
                            if response2.status_code == 200:
                                tmp_dict = response2.json()
                                restaurant_result.append(tmp_dict)
                                search_bool = True
                                print(f'{num}번째 {searchword} 중 {tmp_id}의 밥집(도시락) 요약본 저장 완료')
                            else:
                                print(f'{num}번째 {searchword}로 밥집 요약본 가져오기 실패')
                                sys.exit('response2.status_code 조회해보기')
                            break
                        #카페 베이커리 조회
                        elif '카페' in category:
                            tmp_id = lst['id']
                            #주위 가게로 가져온 id 값으로 요약본 출력
                            url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
                            time.sleep(random.uniform(1.5,2.5))
                            response2 = requests.get(url2,params2,headers=headers2)
                            if response2.status_code == 200:
                                tmp_dict = response2.json()
                                restaurant_result.append(tmp_dict)
                                search_bool = True
                                print(f'{num}번째 {searchword} 중 {tmp_id}의 카페 요약본 저장 완료')
                            else:
                                print(f'{num}번째 {searchword}로 카페 요약본 가져오기 실패')
                                sys.exit('response2.status_code 조회해보기')
                            break
                        elif '베이커리' in category:
                            tmp_id = lst['id']
                            #주위 가게로 가져온 id 값으로 요약본 출력
                            url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
                            time.sleep(random.uniform(1.5,2.5))
                            response2 = requests.get(url2,params2,headers=headers2)
                            if response2.status_code == 200:
                                tmp_dict = response2.json()
                                restaurant_result.append(tmp_dict)
                                search_bool = True
                                print(f'{num}번째 {searchword} 중 {tmp_id}의 밥집(도시락) 요약본 저장 완료')
                            else:
                                print(f'{num}번째 {searchword}로 베이커리 요약본 가져오기 실패')
                                sys.exit('response2.status_code 조회해보기')
                            break
            if search_bool is False:
                none_lst.append(title[num])
                print(f'{num}번째 {searchword}중 음식점 존재X ')

                        
    print(f'{num}번째 {searchword} 종료')

0번째 제주 카페송키 중 1333668157의 1개 요약본 저장 완료
1 번째 그림책방앤카페노란우산 조회 결과 존재 X
2번째 제주 에릭스에스프레소 중 1728932164의 1개 요약본 저장 완료
3번째 제주 일품순두부한림점 중 1284120911의 1개 요약본 저장 완료
4 번째 팔도실비집아라점 조회 결과 존재 X
5번째 제주 봉플라봉뱅 중 1713847517의 1개 요약본 저장 완료
6번째 제주 아라포차 중 1261655201의  술집 요약본 저장 완료
6번째 제주 아라포차 중 1159629995의  술집 요약본 저장 완료
6번째 제주 아라포차 중 528970792의  술집 요약본 저장 완료
6번째 제주 아라포차 중 1948471773의  술집 요약본 저장 완료
6번째 제주 아라포차 종료
7 번째 가빈거래처 조회 결과 존재 X
8번째 제주 스톤아일랜드탭하우스 중 1244422132의 1개 요약본 저장 완료
9 번째 레게펍 조회 결과 존재 X
10번째 제주 옹포분식 중 1992357241의 식당 요약본 저장 완료
10번째 제주 옹포분식 중 1869459211의 식당 요약본 저장 완료
10번째 제주 옹포분식 종료
11번째 제주 연두랑고갈비 중 1604731338의 식당 요약본 저장 완료
11번째 제주 연두랑고갈비 중 1482491837의 식당 요약본 저장 완료
11번째 제주 연두랑고갈비 종료
12 번째 협재더꽃돈본점 조회 결과 존재 X
13번째 제주 우차도리 중 1891477328의 1개 요약본 저장 완료
14 번째 철판구이삼다돈외도점 조회 결과 존재 X
15번째 제주 와르다레스토랑 중 1690028833의 1개 요약본 저장 완료
16번째 제주 길섶나그네 중 20715026의 식당 요약본 저장 완료
16번째 제주 길섶나그네 종료
17 번째 사랑뜰 조회 결과 존재 X
18번째 제주 불티나네 중 1553416534의 1개 요약본 저장 완료
19번째 제주 수니테이블 중 1901662508의 1개 요약본 저장 완료
20번째 제주 연화양꼬치 중 1482063535의 1

133번째 제주 로즈타이거제주흙돼지 중 1068114183의 1개 요약본 저장 완료
134번째 제주 맛있는참세상 중 16873675의 1개 요약본 저장 완료
135 번째 모스트펍 조회 결과 존재 X
136번째 제주 무보 중 1129456863의 식당 요약본 저장 완료
136번째 제주 무보 종료
137번째 제주 이스트포레스트 중 1803296766의 식당 요약본 저장 완료
137번째 제주 이스트포레스트 종료
138번째 제주 콩창고 중 1434338666의 1개 요약본 저장 완료
139번째 제주 초딩입맛 중 1019347726의 1개 요약본 저장 완료
140번째 제주 하르방짬뽕 중 37996260의 1개 요약본 저장 완료
141번째 제주 푸른바다찜탕 중 1546025258의 1개 요약본 저장 완료
142번째 제주 군산야식 중 1497679567의 1개 요약본 저장 완료
143번째 제주 장어창고 중 1023706823의 1개 요약본 저장 완료
144번째 제주 회만원 중 1955853794의 식당 요약본 저장 완료
144번째 제주 회만원 중 73638884의 식당 요약본 저장 완료
144번째 제주 회만원 종료
145번째 제주 동백키친 중 1386240526의 식당 요약본 저장 완료
145번째 제주 동백키친 종료
146 번째 비비큐올리브카페제주시청점 조회 결과 존재 X
147번째 제주 돌담위에흑돼지 중 1144133108의 식당 요약본 저장 완료
147번째 제주 돌담위에흑돼지 종료
148번째 제주 옛날통닭닭치고족치고함덕점 중 1518533451의 1개 요약본 저장 완료
149 번째 골프존클럽하우스그늘집 조회 결과 존재 X
150번째 제주 서머셋 중 1864199529의 식당 요약본 저장 완료
150번째 제주 서머셋 종료
151번째 제주 엘리펀트힙 중 1385242397의 1개 요약본 저장 완료
152번째 제주 소확행 중 1003790165의 1개 요약본 저장 완료
153 번째 나르드카페 조회 결과 존재 X
154 번째 유년기 조회 결과 존재 X
155번째 제주 딱새우랑

257번째 제주 모닥치기아라 중 1435380186의 1개 요약본 저장 완료
258번째 제주 제주돼학교 중 1892672773의 식당 요약본 저장 완료
258번째 제주 제주돼학교 중 32222488의 카페 요약본 저장 완료
258번째 제주 제주돼학교 종료
259 번째 코끼리휴게소점빵 조회 결과 존재 X
260번째 제주 홀릭 중 36244879의 식당 요약본 저장 완료
260번째 제주 홀릭 중 36244890의 카페 요약본 저장 완료
260번째 제주 홀릭 중 38257426의 식당 요약본 저장 완료
260번째 제주 홀릭 중 1430921976의 식당 요약본 저장 완료
260번째 제주 홀릭 중 1808932387의 식당 요약본 저장 완료
260번째 제주 홀릭 중 20117086의  술집 요약본 저장 완료
260번째 제주 홀릭 중 36868786의 식당 요약본 저장 완료
260번째 제주 홀릭 중 1283157147의 식당 요약본 저장 완료
260번째 제주 홀릭 종료
261 번째 협재술집김씨술생활 조회 결과 존재 X
262번째 제주 족발야시장일도점 중 1667265889의 1개 요약본 저장 완료
263번째 제주 명호 중 1503927451의 식당 요약본 저장 완료
263번째 제주 명호 중 1835331356의 식당 요약본 저장 완료
263번째 제주 명호 종료
264번째 제주 김녕에사는김영훈 중 1257655031의 1개 요약본 저장 완료
265번째 제주 카페책자국 중 1472537958의 1개 요약본 저장 완료
266번째 제주 대나무집 중 20111870의 식당 요약본 저장 완료
266번째 제주 대나무집 중 1199129210의 카페 요약본 저장 완료
266번째 제주 대나무집 종료
267번째 제주 카페분식이다 중 1169836420의 1개 요약본 저장 완료
268 번째 달꿈국수 조회 결과 존재 X
269번째 제주 커뮤니티라운지 중 1390406840의 1개 요약본 저장 완료
270번째 제주 역전할머니맥주연동점 중 1029146972의 1개 요약본 저장 완료
271번째

374번째 제주 하음 중 1245656032의 카페 요약본 저장 완료
374번째 제주 하음 종료
375번째 제주 이졸라베르데 중 1280035408의 1개 요약본 저장 완료
376번째 제주 명륜진사갈비삼화점 중 1008683181의 1개 요약본 저장 완료
377번째 제주 사라보쌈족발신제주점 중 1002741354의 1개 요약본 저장 완료
378번째 제주 평대스낵 중 71441654의 1개 요약본 저장 완료
379 번째 맛있는여가생활 조회 결과 존재 X
380번째 제주 상춘재 중 13467718의 1개 요약본 저장 완료
381번째 제주 사노라면 중 1940413003의 식당 요약본 저장 완료
381번째 제주 사노라면 종료
382번째 제주 해녀의부엌 중 1277942900의 식당 요약본 저장 완료
382번째 제주 해녀의부엌 종료
383번째 제주 큰부엌 중 1252752253의 1개 요약본 저장 완료
384번째 제주 봉자네상점 중 1099917791의 1개 요약본 저장 완료
385번째 제주 카페피중 음식점 존재X 
385번째 제주 카페피 종료
386 번째 수제핫바깡스 조회 결과 존재 X
387번째 제주 정부장포차 중 1055204450의 1개 요약본 저장 완료
388번째 제주 연이네 중 1206389982의 식당 요약본 저장 완료
388번째 제주 연이네 중 1967422467의 식당 요약본 저장 완료
388번째 제주 연이네 종료
389 번째 소안뉴이완 조회 결과 존재 X
390번째 제주 그까페 중 1715603019의 식당 요약본 저장 완료
390번째 제주 그까페 중 38411991의 식당 요약본 저장 완료
390번째 제주 그까페 중 1083045545의 식당 요약본 저장 완료
390번째 제주 그까페 중 1840117628의 식당 요약본 저장 완료
390번째 제주 그까페 중 1008522190의 식당 요약본 저장 완료
390번째 제주 그까페 중 1688203652의 식당 요약본 저장 완료
390번째 제주 그까페 중 1407455030의 카페 요약본 저장 완료
390

488번째 제주 노형삼조삼계탕 중 16781853의 1개 요약본 저장 완료
489번째 제주 각지불 중 16888793의 1개 요약본 저장 완료
490번째 제주 어영해녀촌원 중 34555888의 1개 요약본 저장 완료
491번째 제주 노형주점 중 1011585701의  술집 요약본 저장 완료
491번째 제주 노형주점 중 1313832125의  술집 요약본 저장 완료
491번째 제주 노형주점 중 1855561225의  술집 요약본 저장 완료
491번째 제주 노형주점 중 1246477567의  술집 요약본 저장 완료
491번째 제주 노형주점 중 1076332970의  술집 요약본 저장 완료
491번째 제주 노형주점 중 11762357의  술집 요약본 저장 완료
491번째 제주 노형주점 중 37281732의  술집 요약본 저장 완료
491번째 제주 노형주점 중 1329336904의  술집 요약본 저장 완료
491번째 제주 노형주점 중 1090545020의  술집 요약본 저장 완료
491번째 제주 노형주점 중 34016987의  술집 요약본 저장 완료
491번째 제주 노형주점 중 36652727의  술집 요약본 저장 완료
491번째 제주 노형주점 중 1214582835의  술집 요약본 저장 완료
491번째 제주 노형주점 중 1286921711의  술집 요약본 저장 완료
491번째 제주 노형주점 중 1620207662의  술집 요약본 저장 완료
491번째 제주 노형주점 중 1585729847의  술집 요약본 저장 완료
491번째 제주 노형주점 중 1880508332의  술집 요약본 저장 완료
491번째 제주 노형주점 중 1981071280의  술집 요약본 저장 완료
491번째 제주 노형주점 중 1224363557의  술집 요약본 저장 완료
491번째 제주 노형주점 중 1452522820의  술집 요약본 저장 완료
491번째 제주 노형주점 중 11762356의  술집 요약본 저장 완료
491번째 제주 노형주점 종료
492번째 제주 터보 중 31113801의  술집 요약본 

...

694번째 제주 영자포차 중 16692426의 1개 요약본 저장 완료
695번째 제주 영건 중 16835494의 식당 요약본 저장 완료
695번째 제주 영건 종료
696번째 제주 딤섬호프 중 21760466의 1개 요약본 저장 완료
697번째 제주 내가찾은맛고을 중 34269021의 1개 요약본 저장 완료
698번째 제주 김해 중 16755446의 식당 요약본 저장 완료
698번째 제주 김해 중 36948037의 카페 요약본 저장 완료
698번째 제주 김해 중 1798568941의 카페 요약본 저장 완료
698번째 제주 김해 중 1884587649의 카페 요약본 저장 완료
698번째 제주 김해 중 1328812610의 카페 요약본 저장 완료
698번째 제주 김해 중 1219981050의 카페 요약본 저장 완료
698번째 제주 김해 중 1198688000의 카페 요약본 저장 완료
698번째 제주 김해 중 1085259517의 카페 요약본 저장 완료
698번째 제주 김해 중 1686278828의 카페 요약본 저장 완료
698번째 제주 김해 중 31970174의 카페 요약본 저장 완료
698번째 제주 김해 중 12954117의 카페 요약본 저장 완료
698번째 제주 김해 중 32222488의 카페 요약본 저장 완료
698번째 제주 김해 중 1072083628의 카페 요약본 저장 완료
698번째 제주 김해 중 16899342의 카페 요약본 저장 완료
698번째 제주 김해 종료
699번째 제주 해다미 중 16755414의 식당 요약본 저장 완료
699번째 제주 해다미 종료
700번째 제주 우현 중 20243150의 식당 요약본 저장 완료
700번째 제주 우현 종료
701번째 제주 제주동 중 1313217350의 식당 요약본 저장 완료
701번째 제주 제주동 종료
702번째 제주 언양 중 16701712의 1개 요약본 저장 완료
703번째 제주 연동대복관 중 32161831의 1개 요약본 저장 완료
704 번째 쎌빠연동점 조회 결과 존재 X
705번째 제주 팡돌비어 중 

769번째 제주 보고싶다친구야 중 16743353의 1개 요약본 저장 완료
770 번째 르샹스 조회 결과 존재 X
771번째 제주 재연식당 중 455230362의 1개 요약본 저장 완료
772번째 제주 자영 중 16775509의 식당 요약본 저장 완료
772번째 제주 자영 종료
773 번째 윤희 조회 결과 존재 X
774번째 제주 남북통일 중 16776514의 1개 요약본 저장 완료
775번째 제주 명가진숯불갈비 중 1979005209의 1개 요약본 저장 완료
776번째 제주 할머니집 중 16883406의 식당 요약본 저장 완료
776번째 제주 할머니집 중 13016415의 식당 요약본 저장 완료
776번째 제주 할머니집 중 13511954의 카페 요약본 저장 완료
776번째 제주 할머니집 중 1731879961의  술집 요약본 저장 완료
776번째 제주 할머니집 중 38710817의 식당 요약본 저장 완료
776번째 제주 할머니집 중 1109357504의 식당 요약본 저장 완료
776번째 제주 할머니집 중 1032183941의  술집 요약본 저장 완료
776번째 제주 할머니집 중 1519169591의  술집 요약본 저장 완료
776번째 제주 할머니집 중 1076332970의  술집 요약본 저장 완료
776번째 제주 할머니집 중 1029146972의  술집 요약본 저장 완료
776번째 제주 할머니집 중 1331219387의  술집 요약본 저장 완료
776번째 제주 할머니집 중 1526611289의  술집 요약본 저장 완료
776번째 제주 할머니집 중 1872218608의 식당 요약본 저장 완료
776번째 제주 할머니집 중 1249500159의  술집 요약본 저장 완료
776번째 제주 할머니집 중 1117321237의  술집 요약본 저장 완료
776번째 제주 할머니집 중 1356910533의  술집 요약본 저장 완료
776번째 제주 할머니집 중 1781899024의  술집 요약본 저장 완료
776번째 제주 할머니집 중 1195490505의  술집 요약본 저장 완

833번째 제주 탄광촌 중 1276217731의 1개 요약본 저장 완료
834번째 제주 일출바당 중 594106231의 1개 요약본 저장 완료
835번째 제주 카훼리 중 16988905의 1개 요약본 저장 완료
836번째 제주 두리 중 1281341735의 식당 요약본 저장 완료
836번째 제주 두리 중 35579098의 식당 요약본 저장 완료
836번째 제주 두리 중 37190355의 식당 요약본 저장 완료
836번째 제주 두리 중 1372287363의 식당 요약본 저장 완료
836번째 제주 두리 중 1961303573의 식당 요약본 저장 완료
836번째 제주 두리 중 16744594의 식당 요약본 저장 완료
836번째 제주 두리 중 1876477692의 식당 요약본 저장 완료
836번째 제주 두리 종료
837번째 제주 찐국수랑김밥 중 1671520006의 1개 요약본 저장 완료
838번째 제주 산호 중 13166719의 식당 요약본 저장 완료
838번째 제주 산호 중 1833422742의 식당 요약본 저장 완료
838번째 제주 산호 중 1035266908의 식당 요약본 저장 완료
838번째 제주 산호 중 11712122의 식당 요약본 저장 완료
838번째 제주 산호 중 1232388375의 식당 요약본 저장 완료
838번째 제주 산호 중 37405579의 식당 요약본 저장 완료
838번째 제주 산호 중 1800239672의 식당 요약본 저장 완료
838번째 제주 산호 중 38275629의 식당 요약본 저장 완료
838번째 제주 산호 중 1442613576의 식당 요약본 저장 완료
838번째 제주 산호 중 1586864352의 식당 요약본 저장 완료
838번째 제주 산호 중 1574292530의 식당 요약본 저장 완료
838번째 제주 산호 중 16892952의  술집 요약본 저장 완료
838번째 제주 산호 종료
839번째 제주 동광생오리집 중 16697305의 1개 요약본 저장 완료
840번째 제주 산과들비빔밥 중 19891181의 식당 요약본 저장 완료
8

922번째 제주 새한라회관 중 16861676의 1개 요약본 저장 완료
923번째 제주 임춘 중 32155059의 1개 요약본 저장 완료
924번째 제주 바다예찬 중 13444864의 식당 요약본 저장 완료
924번째 제주 바다예찬 종료
925번째 제주 동아리봉개점 중 16769624의 1개 요약본 저장 완료
926번째 제주 으네식당 중 32040830의 1개 요약본 저장 완료
927번째 제주 스카이라운지 중 1890885469의 식당 요약본 저장 완료
927번째 제주 스카이라운지 종료
928번째 제주 씨가 중 16727639의 1개 요약본 저장 완료
929번째 제주 사라 중 16831990의  술집 요약본 저장 완료
929번째 제주 사라 중 1801235666의 식당 요약본 저장 완료
929번째 제주 사라 중 1002741354의 식당 요약본 저장 완료
929번째 제주 사라 종료
930 번째 남사우나스넥 조회 결과 존재 X
931번째 제주 신설돈가 중 19591135의 1개 요약본 저장 완료
932번째 제주 일등본가 중 1426472726의 1개 요약본 저장 완료
933번째 제주 삼도골고루 중 16724440의 1개 요약본 저장 완료
934번째 제주 연수중 음식점 존재X 
934번째 제주 연수 종료
935번째 제주 장원 중 1229647538의 식당 요약본 저장 완료
935번째 제주 장원 중 1366963387의 식당 요약본 저장 완료
935번째 제주 장원 중 20681084의 식당 요약본 저장 완료
935번째 제주 장원 중 1300672462의 식당 요약본 저장 완료
935번째 제주 장원 중 1258978088의 식당 요약본 저장 완료
935번째 제주 장원 중 1686881261의 식당 요약본 저장 완료
935번째 제주 장원 중 1069854984의 식당 요약본 저장 완료
935번째 제주 장원 중 1828308963의 식당 요약본 저장 완료
935번째 제주 장원 중 16879053의 식당 요약본 저장 완료
935번째 제주 장원 중 16696430의 식당 요

KeyboardInterrupt: 

In [19]:
none_lst  = []

In [20]:
with open('./nonelist.txt','r',encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
        line= line.strip()
        none_lst.append(line)

In [22]:
test_df =pd.read_csv('./restaurant_data_raw1.csv')

In [29]:
test_df['fullAddress'].isin(['제주특별자치도'])

0        False
1        False
2        False
3        False
4        False
         ...  
12664    False
12665    False
12666    False
12667    False
12668    False
Name: fullAddress, Length: 12669, dtype: bool

In [30]:
restaurant_result = []

In [31]:
for num in range(7383,len(title)):
    searchword = '제주 '+  title[num]
    #정규표현식으로 ( ) 안 텍스트 전부 삭제
    rmve_bracket = r'\([^)]*\)'
    searchword = re.sub(rmve_bracket,'',searchword)
    params['query'] = searchword
    time.sleep(random.uniform(1.5,2.5))
    response1 = requests.get(url,params,headers=headers)
    if response1.status_code == 200:
        j_text = response1.json()
    else:
        print(f'{num}번째 실패')
        sys.exit('response1.status_code 조회해보기')
        
    #장소정보 저장    
    tmp_places = j_text['result']['place']    
    search_bool = False
    #검색결과 없을 때
    if tmp_places is None:
        none_lst.append(title[num])
        print(f'{num} 번째 {title[num]} 조회 결과 존재 X')
        continue
    #검색 결과 존재시    
    else:
        tmp_list=tmp_places['list']
        if tmp_list is None:
            none_lst.append(title[num])
            print(f'{num} 번째 {title[num]} 조회 결과 존재 X')
            continue
        else:
            
            #검색결과가 하나일 때 바로 조회
            if len(tmp_list) == 1:
                tmp_id = tmp_list[0]['id']
                url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
                time.sleep(random.uniform(1.5,2.5))
                response2 = requests.get(url2,params2,headers=headers2)
                if response2.status_code == 200:
                    tmp_dict = response2.json()
                    restaurant_result.append(tmp_dict)
                    search_bool = True
                    print(f'{num}번째 {searchword} 중 {tmp_id}의 1개 요약본 저장 완료')
                else:
                    print(f'{num}번째 {searchword}의 {tmp_id}로 1개 요약본 가져오기 실패')
                    sys.exit('response2.status_code 조회해보기')
                continue
            #검색결과 여러개일 때    
            else:
                for index, lst in enumerate(tmp_list):
                    tmp_categories =lst['category']
                    for category in tmp_categories:
                        if '식' in category:
                            tmp_id = lst['id']
                            #주위 가게로 가져온 id 값으로 요약본 출력
                            url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
                            time.sleep(random.uniform(1.5,2.5))
                            response2 = requests.get(url2,params2,headers=headers2)
                            if response2.status_code == 200:
                                tmp_dict = response2.json()
                                restaurant_result.append(tmp_dict)
                                search_bool = True
                                print(f'{num}번째 {searchword} 중 {tmp_id}의 식당 요약본 저장 완료')
                            else:
                                print(f'{num}번째 {searchword}의 {tmp_id}로 식당 요약본 가져오기 실패')
                                sys.exit('response2.status_code 조회해보기')
                            break
                        elif '술집' in category:
                            tmp_id = lst['id']
                            #주위 가게로 가져온 id 값으로 요약본 출력
                            url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
                            time.sleep(random.uniform(1.5,2.5))
                            response2 = requests.get(url2,params2,headers=headers2)
                            if response2.status_code == 200:
                                tmp_dict = response2.json()
                                restaurant_result.append(tmp_dict)
                                search_bool = True
                                print(f'{num}번째 {searchword} 중 {tmp_id}의  술집 요약본 저장 완료')
                            else:
                                print(f'{num}번째 {searchword}의 {tmp_id}로 술집 요약본 가져오기 실패')
                                sys.exit('response2.status_code 조회해보기')
                            break
                        elif '밥' in category:
                            tmp_id = lst['id']
                            #주위 가게로 가져온 id 값으로 요약본 출력
                            url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
                            time.sleep(random.uniform(1.5,2.5))
                            response2 = requests.get(url2,params2,headers=headers2)
                            if response2.status_code == 200:
                                tmp_dict = response2.json()
                                restaurant_result.append(tmp_dict)
                                search_bool = True
                                print(f'{num}번째 {searchword} 중 {tmp_id}의 밥집(도시락) 요약본 저장 완료')
                            else:
                                print(f'{num}번째 {searchword}로 밥집 요약본 가져오기 실패')
                                sys.exit('response2.status_code 조회해보기')
                            break
                        #카페 베이커리 조회
                        elif '카페' in category:
                            tmp_id = lst['id']
                            #주위 가게로 가져온 id 값으로 요약본 출력
                            url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
                            time.sleep(random.uniform(1.5,2.5))
                            response2 = requests.get(url2,params2,headers=headers2)
                            if response2.status_code == 200:
                                tmp_dict = response2.json()
                                restaurant_result.append(tmp_dict)
                                search_bool = True
                                print(f'{num}번째 {searchword} 중 {tmp_id}의 카페 요약본 저장 완료')
                            else:
                                print(f'{num}번째 {searchword}로 카페 요약본 가져오기 실패')
                                sys.exit('response2.status_code 조회해보기')
                            break
                        elif '베이커리' in category:
                            tmp_id = lst['id']
                            #주위 가게로 가져온 id 값으로 요약본 출력
                            url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
                            time.sleep(random.uniform(1.5,2.5))
                            response2 = requests.get(url2,params2,headers=headers2)
                            if response2.status_code == 200:
                                tmp_dict = response2.json()
                                restaurant_result.append(tmp_dict)
                                search_bool = True
                                print(f'{num}번째 {searchword} 중 {tmp_id}의 밥집(도시락) 요약본 저장 완료')
                            else:
                                print(f'{num}번째 {searchword}로 베이커리 요약본 가져오기 실패')
                                sys.exit('response2.status_code 조회해보기')
                            break
            if search_bool is False:
                none_lst.append(title[num])
                print(f'{num}번째 {searchword}중 음식점 존재X ')

                        
    print(f'{num}번째 {searchword} 종료')

7383번째 제주 쉼터중 음식점 존재X 
7383번째 제주 쉼터 종료
7384 번째 베스트프렌드 조회 결과 존재 X
7385번째 제주 삼다도횟집 중 12883219의 식당 요약본 저장 완료
7385번째 제주 삼다도횟집 중 20668445의 식당 요약본 저장 완료
7385번째 제주 삼다도횟집 종료
7386번째 제주 제주돈오리협재점 중 1465466265의 1개 요약본 저장 완료
7387번째 제주 현미 중 1893918229의 식당 요약본 저장 완료
7387번째 제주 현미 중 37577180의 식당 요약본 저장 완료
7387번째 제주 현미 중 1880638969의 식당 요약본 저장 완료
7387번째 제주 현미 중 1920052669의 밥집(도시락) 요약본 저장 완료
7387번째 제주 현미 중 1150750099의 식당 요약본 저장 완료
7387번째 제주 현미 중 696262903의 식당 요약본 저장 완료
7387번째 제주 현미 중 1406719190의 식당 요약본 저장 완료
7387번째 제주 현미 중 1055942342의 식당 요약본 저장 완료
7387번째 제주 현미 중 1653305908의 식당 요약본 저장 완료
7387번째 제주 현미 중 1488098260의 식당 요약본 저장 완료
7387번째 제주 현미 중 1276408866의 식당 요약본 저장 완료
7387번째 제주 현미 중 1173904475의 식당 요약본 저장 완료
7387번째 제주 현미 종료
7388 번째 황토우렁이 조회 결과 존재 X
7389 번째 물방포차 조회 결과 존재 X
7390번째 제주 조근아지망 중 16874968의 1개 요약본 저장 완료
7391번째 제주 오름해산물 중 345750987의 식당 요약본 저장 완료
7391번째 제주 오름해산물 중 1984039959의 식당 요약본 저장 완료
7391번째 제주 오름해산물 종료
7392 번째 제주밥보장터 조회 결과 존재 X
7393번째 제주 일도촌 중 16695080의 1개 요약본 저장 완료
7394번째 제주 국수몰고랑 중 38710871의 1개 

: 

: 

In [76]:
tmp_df = pd.DataFrame(restaurant_result)

In [77]:
tmp_df.to_csv('../data/naver_crawling/restaurant_data_raw1.csv',index=False)

In [78]:
with open('./nonelist.txt','w',encoding='UTF-8') as f:
    for lst in none_lst:
        f.write(lst+'\n')


In [ ]:
none_df =jeju_food[jeju_food['사업자명'].isin(none_lst)]

In [ ]:
none_df.info()

In [79]:
searchword

'제주 행즈'